In [8]:
import cv2
import face_recognition
import pickle
import os
from datetime import datetime
from openpyxl import Workbook, load_workbook
import numpy as np
from tabulate import tabulate

with open("encoded file.p", 'rb') as file:
    encodelistknown, ids = pickle.load(file)

excel_file = "attendance.xlsx"
if os.path.exists(excel_file):
    wb = load_workbook(excel_file)
    ws = wb.active
else:
    wb = Workbook()
    ws = wb.active
    ws.append(["ID", "Date", "Time"])

today_str = datetime.now().strftime("%Y-%m-%d")

# Session-only attendance tracking
session_attendance = set()  # Only track who was recognized *this session*
attendance_data = []        # Store tuples of (ID, Date, Time)

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 720)
cap.set(4, 720)

print("📸 Starting Face Recognition Attendance...")
print("➡ Press 's' to save attendance")
print("➡ Press 'p' to stop session\n")

while True:
    r, f = cap.read()
    if not r:
        print("⚠️ Webcam not accessible.")
        break

    imgS = cv2.resize(f, (0, 0), fx=0.25, fy=0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    loc = face_recognition.face_locations(imgS)
    face_encodings = face_recognition.face_encodings(imgS, loc)

    for face_encoding, face_location in zip(face_encodings, loc):
        dis = face_recognition.face_distance(encodelistknown, face_encoding)
        matches = face_recognition.compare_faces(encodelistknown, face_encoding)

        if len(dis) > 0:
            best_match_index = np.argmin(dis)
            if matches[best_match_index]:
                matched_id = ids[best_match_index]
                name = matched_id.upper()

                if matched_id not in session_attendance:
                    session_attendance.add(matched_id)
                    now = datetime.now()
                    date_str = now.strftime("%Y-%m-%d")
                    time_str = now.strftime("%H:%M")
                    attendance_data.append([name, date_str, time_str])
                    print(f"✅ Recognized and marked: {name}")


                top, right, bottom, left = [v * 4 for v in face_location]
                cv2.rectangle(f, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(f, name, (left, top - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    cv2.imshow("Face Recognition Attendance", f)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("p"):
        break
    elif key == ord("s"):
        if attendance_data:
            for row in attendance_data:
                ws.append(row)
            wb.save(excel_file)
            print(f"\n💾 Attendance saved to {excel_file}\n")
        else:
            print("⚠️ No attendance to save.")
cap.release()
cv2.destroyAllWindows()


if attendance_data:
    print("\n📋 Attendance This Session:")
    print(tabulate(attendance_data, headers=["ID", "Date", "Time"], tablefmt="fancy_grid"))
else:
    print("\n⚠️ No attendance recorded this session.")




📸 Starting Face Recognition Attendance...
➡ Press 's' to save attendance
➡ Press 'p' to stop session

✅ Recognized and marked: PAWAN KALYAN
✅ Recognized and marked: PRABHAS
✅ Recognized and marked: RAM CHARAN
✅ Recognized and marked: RAVI TEJA
✅ Recognized and marked: MAHESH BABU
✅ Recognized and marked: NTR
✅ Recognized and marked: ALLU ARJUN
✅ Recognized and marked: NITHIN

💾 Attendance saved to attendance.xlsx


📋 Attendance This Session:
╒══════════════╤════════════╤════════╕
│ ID           │ Date       │ Time   │
╞══════════════╪════════════╪════════╡
│ PAWAN KALYAN │ 2025-09-19 │ 18:40  │
├──────────────┼────────────┼────────┤
│ PRABHAS      │ 2025-09-19 │ 18:40  │
├──────────────┼────────────┼────────┤
│ RAM CHARAN   │ 2025-09-19 │ 18:40  │
├──────────────┼────────────┼────────┤
│ RAVI TEJA    │ 2025-09-19 │ 18:40  │
├──────────────┼────────────┼────────┤
│ MAHESH BABU  │ 2025-09-19 │ 18:40  │
├──────────────┼────────────┼────────┤
│ NTR          │ 2025-09-19 │ 18:40  │
├───────